# NLP Sentiment Analysis with RoBERTa

Train and evaluate a RoBERTa model for sentiment analysis using NLTK's movie_reviews corpus (2,000 reviews labeled as "positive" or "negative").

RoBERTa (Robustly Optimized BERT Approach) is a state-of-the-art transformer model that outperforms traditional machine learning approaches.

### Data Pre-Processing

In [ ]:
# Import libraries
# nltk.download('movie_reviews') # Download corpus if not already done
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split

# Load movie reviews as raw text
texts = []
labels = []

# RoBERTa works well with raw text, but needs binary labels
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        text = ' '.join(movie_reviews.words(fileid))
        texts.append(text)
        # Convert label to binary: 1 = positive, 0 = negative
        labels.append(1 if category == 'pos' else 0)

# Check results, displaying sample text and label
print(f"Loaded {len(texts)} movie reviews")
print(f"Example text: {texts[0][:50]}...")
print(f"Example label: {labels[0]} ({'positive' if labels[0] == 1 else 'negative'})")

In [ ]:
# Split into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, # raw text of movie review
    labels, # binary label
    test_size=0.2, # reserve 20% for testing
    random_state=113 # set for reproducibility
)

In [ ]:
# Calculate descriptive statistics for train_texts (i.e. review lengths)
# RoBERTa can handle up to 512 tokens, but using fewer tokens speeds up training
train_lengths = [len(text.split()) for text in train_texts]
mean_length = sum(train_lengths) / len(train_lengths)
median_length = sorted(train_lengths)[len(train_lengths) // 2]
std_length = (sum((x - mean_length) ** 2 for x in train_lengths) / len(train_lengths)) ** 0.5
min_length = min(train_lengths)
max_length = max(train_lengths)

# Display results
print(f"Median review length (in tokens): {median_length:.2f}")
print(f"Mean review length (in tokens): {mean_length:.2f}")
print(f"SD review length (in tokens): {std_length:.2f}")
print(f"Min review length (in tokens): {min_length}")
print(f"Max review length (in tokens): {max_length}")

### Model Training

- Tokenize text, converting from raw text to token IDs.
- Create a PyTorch Dataset, wrapping tokenized text so RoBERTa can use it.
- Load the pre-trained RoBERTa model (roberta-base).
- Specify training arguments, like number of epochs.
- Calculate performance metrics (accuracy, precision, recall, F1) per epoch.

In [ ]:
# Import libraries for RoBERTa
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained RoBERTa tokenizer
model_name = "roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Tokenize text, converting text into numbers (token IDs) that RoBERTa can understand
# Each word is converted to a number (e.g. "the" to a token ID like 123)
# Each document (review) is truncated or padded to 512 tokens max (see above descriptive statistics)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [ ]:
# Create a PyTorch Dataset class
# Convert data to PyTorch tensors (numerical arrays optimized for GPU processing)
# This enables batching during training (see training arguments below)
import torch

class MovieReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add labels (0|1 sentiment labels) as a tensor
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = MovieReviewDataset(train_encodings, train_labels)
test_dataset = MovieReviewDataset(test_encodings, test_labels)

In [ ]:
# Load the pre-trained RoBERTa model (num_labels = 2 to categorize positive/negative)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# Set up training configuration
import warnings
warnings.filterwarnings('ignore', message='.*pin_memory.*') # Ignore pinned memory warnings
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Specify training arguments like number of training epochs
training_args = TrainingArguments(
    num_train_epochs=5,              # Train for 5 complete passes through the data
    per_device_train_batch_size=10,  # Process 10 reviews at a time during training
    per_device_eval_batch_size=15,   # Process 15 reviews at a time during evaluation
    warmup_steps=500,                # Gradually increase learning rate for first 500 steps
    weight_decay=0.01,               # Prevent overfitting
    logging_steps=100,               # Log training progress every 100 steps
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save model at the end of each epoch
    load_best_model_at_end=True,     # Load the best model when finished
)

# Define performance metrics function
def performance_metrics(pred):
    """Calculate performance metrics"""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args, # See above
    train_dataset=train_dataset,# Train on training set
    eval_dataset=test_dataset, # Evaluate on test set
    compute_metrics=performance_metrics # See above
)

### Model Evaluation

In [ ]:
# Go get a coffee while the model is training...
trainer.train()

### Visualization

In [ ]:
# Import plotting library
import plotnine as plt
import pandas as pd

# Convert log history to DataFrame and filter for evaluation metrics
log_df = pd.DataFrame(trainer.state.log_history)
eval_df = log_df[log_df['eval_accuracy'].notna()].copy()

# Create plot
plot = (plt.ggplot(eval_df, plt.aes(x='epoch', y='eval_accuracy')) +
    plt.geom_line(size=1.5, color='darkblue') +
    plt.geom_point(size=3, color='darkblue') +
    # Set titles and labels
    plt.ggtitle('RoBERTa Accuracy Over Epochs') +
    plt.xlab('Epoch') +
    plt.ylab('Accuracy') +
    # Customize theme
    plt.theme_bw(base_family = "Arial", base_size = 12) +
    plt.theme(figure_size=(6.5, 4))
)

# Display plot
display(plot)